In [ ]:
dist_notebook_path = ""
input_files= ""
#WARNING: When re-running the notebook for audit, change the dist_notebook_path below to "./output_praiseDistribution.ipynb"
#then go to "Cell > Run all" -- This only works for the notebook in 
#"distribution_results/round ?/results/analysis_outputs/output_general_RAD_report.ipynb"

In [ ]:
from ipyfilechooser import FileChooser

import os
import sys
import inspect

import pandas as pd 
import numpy as np 

#this is mainly for when we re-run the notebook (TO DO: redo when file structure finalized)
dir2 = os.path.abspath('../../../../analysis_tools')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)
    

from analysis_tools.module_libraries import general_tool_module as tools
from analysis_tools.module_libraries import praise_tool_module as praise_tools


import holoviews as hv
from holoviews import opts
import plotly.graph_objects as go
import plotly.express as px

import base64
from IPython.display import HTML
from IPython.display import Markdown as md

import scrapbook as sb

#fc_praise = FileChooser('./')
#fc_sourcecred = FileChooser('./')
#fc_rewardboard = FileChooser('./')

#print("== Please choose the Praise CSV file == ")
#display(fc_praise)
#print("== Please choose the Sourcecred CSV file == ")
#display(fc_sourcecred)
#print("== Please choose the Rewardboard address list CSV file == ")
#display(fc_rewardboard)

In [ ]:
#getting the data from the previous notebook

nb = sb.read_notebook(dist_notebook_path)

CROSS_PERIOD_ROOT = input_files["cross_period_root"]

final_token_allocations = nb.scraps['final_token_allocations'].data
rewardboard_rewards = nb.scraps['rewardboard_rewards'].data
quantifier_rewards = nb.scraps['quantifier_rewards'].data
quantifier_rating_table = nb.scraps['quantifier_rating_table'].data

processed_praise = nb.scraps['processed_praise'].data
praise_by_user = nb.scraps['praise_by_user'].data
#processed_sourcecred = nb.scraps['processed_sourcecred'].data

#sourcecred_distribution = nb.scraps['sourcecred_distribution'].data
praise_distribution = nb.scraps['praise_distribution'].data

DISTRIBUTION_NAME = nb.scraps['distribution_name'].data
TOTAL_TOKENS_ALLOCATED = nb.scraps['total_tokens_allocated'].data
DUPLICATE_PRAISE_WEIGHT = nb.scraps['praise_quantify_duplicate_praise_valuation'].data
NUMBER_OF_QUANTIFIERS_PER_PRAISE = nb.scraps['quantifiers_per_praise'].data
PERIOD_START_DATE = praise_distribution['DATE'].min()[:10]
PERIOD_END_DATE = praise_distribution['DATE'].max()[:10]
PSEUDONYMS_USED = nb.scraps['pseudonyms_used'].data

pseudonym_string = "were"  if bool(PSEUDONYMS_USED) else "were not"




In [ ]:
hv.extension('bokeh', logo=False)

# Rewards Analytics and Distribution Dashboard for Quantification Review
This goal of this document is to offer an easy way to process the outputs of the praise reward system and perform an analysis of the resulting token reward distribution. It should be considered a work-in-progress. 

In [ ]:
md(f"<h2>Distribution report for {DISTRIBUTION_NAME}</h2> \
<ul><li>This period covers praise given between <b>{PERIOD_START_DATE}</b> and  <b>{PERIOD_END_DATE}</b>. </li> \
<li> We allocated a total of <b>{TOTAL_TOKENS_ALLOCATED}</b> TEC tokens for rewards. </li>\
<li>Duplicate praise received a weighting of <b>{DUPLICATE_PRAISE_WEIGHT}</b> the value of the original praise. </li> \
<li>We assigned <b>{NUMBER_OF_QUANTIFIERS_PER_PRAISE}</b> quantifiers per praise instance. </li> \
<li>Praise receiver names <b>{pseudonym_string}</b> hidden behind pseudonyms during quantification </li> \
</ul>")

# Praise Data Visualization



### Rating distribution
Since praise gets valued on a scale, we can take a look at how often each value of the scale gets assigned by quantifiers.
Note: how to process the duplicate scores? For now, just delete them.


In [ ]:
# clear out the quantifiers who didn't give any rating (i.e. all scores are 0)
quantifier_sum = quantifier_rating_table[['QUANT_ID','QUANT_VALUE']].groupby('QUANT_ID').sum()
norating_quantifiers = quantifier_sum.loc[quantifier_sum['QUANT_VALUE']==0].index.tolist()

In [ ]:
freq = quantifier_rating_table[['QUANT_VALUE']].value_counts().rename_axis('QUANT_VALUE').reset_index(name='counts').sort_values(by=['QUANT_VALUE'])
freq['QUANT_VALUE'] = freq['QUANT_VALUE'].astype('string')

fig_freq = px.bar(freq, x="QUANT_VALUE", y="counts", labels={"QUANT_VALUE": "Rating","counts": "Number of appearances"}, title="Praise Rating Distribution", width=800, height=300)
fig_freq


### TOP10 highest rating praise

In [ ]:
toppraise = praise_distribution.sort_values(by=['AVG SCORE'],ascending=False).iloc[:10]
for kr,row in toppraise.iterrows():
    from_user = row['FROM USER ACCOUNT']
    to_user = row['TO USER ACCOUNT']
    reason = row['REASON']
    score = row['AVG SCORE']
    
    print(f'Praise score average: {score}\nFROM {from_user} TO {to_user},reason:\n{reason}\n')

### Praise Reward Distribution

We can now take a look at the distribution of the received praise rewards. You can toggle the inclusion of the different sources by clicking on the legend. 

In [ ]:
#pr_distribution = praise_by_user[['USER IDENTITY', 'PERCENTAGE']].sort_values(by=['PERCENTAGE'], ascending=False)

#fig_pr_distribution = px.bar(pr_distribution, x='USER IDENTITY', y='PERCENTAGE', labels={"IDENTITY": "User","PERCENTAGE": "% of total"}, title="Praise Reward Distribution")#.opts(width=800, height=500, title='SourceCred Distribution', xlabel='Value', ylabel='% of Total', xaxis='bare')
#fig_pr_distribution.update_xaxes(showticklabels=False)

#fig_pr_distribution


In [ ]:
#TO DO: show usernames on the x axis
#fig_final_alloc = px.bar(final_token_allocations, x="USER IDENTITY", y = ["QUANT_REWARD", "REWARDBOARD_REWARD", "PRAISE_REWARD", "SOURCECRED_REWARD"], title="Rewards received by category", color_discrete_map = {'PRAISE_REWARD': 'blue', 'SOURCECRED_REWARD': 'red', 'QUANT_REWARD':'green', 'REWARDBOARD_REWARD':'yellow'})
fig_final_alloc = px.bar(final_token_allocations, x="USER IDENTITY", y = ["QUANT_REWARD", "REWARDBOARD_REWARD", "PRAISE_REWARD"], title="Rewards received by category", color_discrete_map = {'PRAISE_REWARD': 'blue', 'QUANT_REWARD':'green', 'REWARDBOARD_REWARD':'yellow'})
fig_final_alloc.update_xaxes(showticklabels=False)
fig_final_alloc


### Praise Giving Distribution
We can also take a look at the distribution of the people giving praise. 


In [ ]:
#first we calculate the individual contributions of each praise giver
praise_by_giver = praise_distribution[['FROM USER ACCOUNT', 'AVG SCORE', 'PERCENTAGE', 'TOKEN TO RECEIVE']].copy().groupby(['FROM USER ACCOUNT']).agg('sum').reset_index()
praise_by_giver.rename(columns= {'TOKEN TO RECEIVE': 'TOKENS GAVE'}, inplace = True)
praise_by_giver.sort_values(by='TOKENS GAVE',inplace=True,ascending=False)
#praise_by_giver.style

fig_praisegiver = px.bar(praise_by_giver, x="FROM USER ACCOUNT",y='TOKENS GAVE',title='Praise Giver Sorted by Total Score')
fig_praisegiver.show()

### Praise Flows

Now for something more fun: let's surface the top "praise flows" from the data. Thanks to @inventandchill for this awesome visualization! 
On one side we have the top 20 praise givers separately (modifiable by changing the variable n_senders), on the other the top 25 receivers (modifiable by changing the variable n_receivers). The people outside the selection get aggregated into the "REST FROM" and "REST TO" categories.

In [ ]:
NUMBER_OF_SENDERS_FLOW = 15 #The left side, the praise senders. X largest ones + one bucket for the rest 
NUMBER_OF_RECEIVERS_FLOW = 25 #The right side, the praise receivers. X largest ones + one bucket for the rest 
dist_for_praise_flow = praise_distribution.rename(columns = {'FROM USER ACCOUNT':'FROM', 'TO USER ACCOUNT':'TO'})
praise_flow = praise_tools.prepare_praise_flow(dist_for_praise_flow.copy(), n_senders=NUMBER_OF_SENDERS_FLOW, n_receivers=NUMBER_OF_RECEIVERS_FLOW)
#praise_flow

In [ ]:
%%opts Sankey (cmap='Category10' edge_color='FROM' edge_line_width=0 node_alpha=1.0)
%%opts Sankey [node_sort=False label_position='outer' bgcolor="snow" node_width=40 node_sort=True ]
%%opts Sankey [width=1000 height=800 title="Praise flow for Batch 1. Sum of Praise. Left - praise sender. Right - praise receiver"]
%%opts Sankey [margin=0 padding=0 show_values=True]

hv.Sankey(praise_flow, kdims=["FROM", "TO"], vdims=["AVG SCORE"])

# Quantifier Data
Let's take a closer look at the quantification process and and see if we can spot any problems:


## Praise Outliers
To aid the revision process, we highlight disagreements between quantifiers.

### Outliers sort by spreads

Here we generate a table which sorts the praise by the size of the spread between the highest and lowest scores. It gives us an overview of the spread distribution. <br>

For an exhaustive list, take a look at the exported file "praise_outliers.csv" .

In [ ]:


#for general use
col_dismissed = [f'DISMISSED {k+1}' for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]
col_dupids = [f'DUPLICATE ID {k+1}' for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]
col_scores= [f'SCORE {k+1}' for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]

#clean the Dataframe and remove praise where there is dismissal agreement
praisecheck_df = praise_distribution.drop(['TO ETH ADDRESS', 'TO USER ACCOUNT ID', 'FROM ETH ADDRESS', 'FROM USER ACCOUNT ID', 'SOURCE ID', 'SOURCE NAME', 'PERCENTAGE', 'TOKEN TO RECEIVE'], axis=1)
ethadds = [f'QUANTIFIER {k+1} ETH ADDRESS' for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]
praisecheck_df.drop(ethadds,axis=1,inplace=True)

praisecheck_clean_controversial = praisecheck_df.loc[praisecheck_df[col_dismissed].sum(axis=1)<NUMBER_OF_QUANTIFIERS_PER_PRAISE,:]


#scores = ['SCORE '+str(k+1) for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]
#praisecheck_df['SPREAD'] = praisecheck_df[scores].max(axis=1) - praisecheck_df[scores].min(axis=1)
#sort_by_controversial = praisecheck_df.sort_values(by= 'SPREAD', ascending = False).reset_index()

# Saves the outlier table in an external file for easier review
#praise_outliers_csv = sort_by_controversial.to_csv(index=False)
#with open('praise_outliers.csv', 'w') as f:
#    f.write(praise_outliers_csv)



In [1]:
#reinstate the original scores for the duplicates before calculating the spread
dupclean_praisecheck = praisecheck_clean_controversial.copy()
for i, row in praisecheck_clean_controversial.iterrows():
    for j, dup_id_label in enumerate(col_dupids):
        #print(f'Label: {dup_id_label} + Row: {row[dup_id_label]} + "NaN: {np.nan}')
        if row[dup_id_label] is not None:
            #find the score   	   
            find_value = praisecheck_df.loc[praisecheck_df['ID']==row[dup_id_label], col_scores[j]]

            #FOR DEBUG:  
            try:
                #substitute it in dupclean 
                dupclean_praisecheck.at[i, str(col_scores[j])] = int(find_value)  
            except:
                #account for the bug in early rounds
                dupclean_praisecheck.at[i, str(col_scores[j])] = 0 

NameError: name 'praisecheck_clean_controversial' is not defined

In [ ]:
#discard no-shows (score = 0 and not dismissed, after the above check for duplicates) and calculate spread
def noShow(a,b):
    if int(a)==0 and bool(b)==False: 
        return np.nan
    else:
        return a


for i,  score_col in enumerate(col_scores):
    dupclean_praisecheck[score_col] = dupclean_praisecheck.apply(lambda x: noShow(x[score_col], x[col_dismissed[i]]) , axis=1) 

    
dupclean_praisecheck['SPREAD'] = dupclean_praisecheck[col_scores].max(axis=1) - dupclean_praisecheck[col_scores].min(axis=1)
sort_by_controversial = dupclean_praisecheck.sort_values(by= 'SPREAD', ascending = False).reset_index()

In [ ]:
# Save the outlier table in an external file for easier review
praise_outliers_csv = sort_by_controversial.to_csv(index=False)
with open('praise_outliers.csv', 'w') as f:
    f.write(praise_outliers_csv)

In [ ]:
# remove outliers if there's dimissal 
#col_dismissed = [f'DISMISSED {k+1}' for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]

#clean_controversial = sort_by_controversial.loc[sort_by_controversial[col_dismissed].sum(axis=1)==0,:]

# remove rows when there's duplication -- to be done, better deal with duplication
#col_dupids = [f'DUPLICATE ID {k+1}' for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]
#col_scores= [f'SCORE {k+1}' for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]

#dupclean_controversial = clean_controversial.loc[[set(row[col_dupids].values)=={None} for kr,row in clean_controversial.iterrows()]]



In [ ]:
# temp: instead of remove duplications, use the original score to replace the duplicated score.
# but there seems to be some problem in the current datasheet that this thing doesn't work right...
#col_dupids = [f'DUPLICATE ID {k+1}' for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]
#col_quants= [f'QUANTIFIER {k+1} USERNAME' for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]
#col_scores= [f'SCORE {k+1}' for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]

#dupclean_controversial = clean_controversial.copy()
#for kr, row in clean_controversial.iterrows():
#    for kq,kcol_dup in enumerate(col_dupids):
#        if row[kcol_dup] is not None:
#            # find the score of the original 
#            duprow = sort_by_controversial.loc[sort_by_controversial['ID']==row[kcol_dup]]
#            row_quant_id = duprow[col_quants].values== row[col_quants[kq]]
#            dupscore = duprow[np.array(col_scores)[row_quant_id[0]]]
            
            # copy that into the duplicated praise as the score
#            dupclean_controversial.at[kr,col_scores[kq]]= dupscore.values[0][0]
            

In [ ]:
# remove some columns for easier visual examination
#simp_controversial = dupclean_controversial.drop(col_dupids+col_dismissed,axis='columns')
#simp_controversial.to_csv('cleaned_praise_outliers.csv')

In [ ]:
# look at what kinda messages get higher spread 
sort_by_controversial['MAX SCORE'] = sort_by_controversial[col_scores].max(axis=1)
spread_df = sort_by_controversial[['REASON','AVG SCORE','MAX SCORE','SPREAD']].sort_values(by='SPREAD')
px.scatter(spread_df,y='MAX SCORE',x='SPREAD',hover_name='REASON')

In [ ]:
# sort_by_controversial.head(20) # top 20 spreads

This is a visual aid. ATTENTION! If there are several praise instances with similar spread and quant score, all but one end up "hidden" on the chart. 

In [ ]:
#fig_spread = px.scatter(sort_by_controversial, x="AVG SCORE", y="SPREAD", hover_data=[sort_by_controversial.index, 'ID'])
#fig_spread

## Praise score by quantifier -- outliers among the quantifiers?

Let's see the range of praise scores every quantifier gave to see the behavior difference of quantifiers.

To interpret the box plot:

- Bottom horizontal line of box plot is minimum value

- First horizontal line of rectangle shape of box plot is First quartile or 25%

- Second horizontal line of rectangle shape of box plot is Second quartile or 50% or median.

- Third horizontal line of rectangle shape of box plot is third quartile or 75%

- Top horizontal line of rectangle shape of box plot is maximum value.




In [ ]:
quant_boxplot = quantifier_rating_table[['QUANT_ID', 'QUANT_VALUE']].copy()
fig_box = px.box(quant_boxplot, x="QUANT_ID", y="QUANT_VALUE", points=False)
fig_box

In [ ]:
from analysis_tools.module_libraries.praise_analyis import praise_round
pr = praise_round(praisedata=praise_distribution,quantifiertable=quantifier_rating_table)


### score displacement: tendency to under/over-scoring?

In [ ]:
fig=pr.plot_mean_displacement()
fig.show()

### scoring correlation: how similiar am i scoring with others?

In [ ]:
fig=pr.plot_coefficient()
fig.show()

## Agreement on duplication

In [ ]:
# pre-process to remove None
dup_agree_check = dupclean_praisecheck.copy()

for k in range(1,NUMBER_OF_QUANTIFIERS_PER_PRAISE+1):
    col = 'DUPLICATE ID '+str(k)
    dup_agree_check.loc[dup_agree_check[col].isnull(),col]=0

dup_agree_check['DUPLICATION AGREED'] = [len(set(kk))==1 for kk in dup_agree_check[col_dupids].values]

In [ ]:
# output the duplication disagreement
duplication_disagreement = dup_agree_check.loc[dup_agree_check['DUPLICATION AGREED']!=True,:]

duplication_disagreement = duplication_disagreement.drop(['SPREAD']+col_scores+col_dismissed, axis=1)

print(f'Among {len(dup_agree_check)} praises, {len(duplication_disagreement)} ({len(duplication_disagreement)/len(praisecheck_df)*100:.2f}%) do not agree on duplication')

In [ ]:
#col_dup = [f'DUPLICATE MSG {k+1}' for k in range(NUMBER_OF_QUANTIFIERS_PER_PRAISE)]
#duplication_disagreement[col_dup] = '/'
#unfound_dupid=[]
#for kr,row in duplication_disagreement.iterrows():
#    for k in range(1,NUMBER_OF_QUANTIFIERS_PER_PRAISE+1):
#        dup_id=row['DUPLICATE ID '+str(k)]
#        if dup_id!=0:
#            try:
#                duplication_disagreement['DUPLICATE MSG '+str(k)].at[kr] = praisecheck_df.loc[praisecheck_df['ID']==dup_id,'REASON'].values[0]
#            except:
#                if len(praisecheck_df.loc[praisecheck_df['ID']==dup_id,'REASON'].values)==0:
#                    unfound_dupid.append(dup_id)
#                    duplication_disagreement['DUPLICATE MSG '+str(k)].at[kr] = 'duplication not found'
#if len(unfound_dupid)>0:
#    print(f'{len(unfound_dupid)} duplication ids not found!')


Praise instances with disagreements in duplication are collected in 'results/duplication_examination.csv'. To compare, look at the last 4 columns: 'DUPLICATE MSG 1/2/3' and 'ORIGINAL MSG'.

In [ ]:
duplication_disagreement['ORIGINAL MSG']=duplication_disagreement['REASON'] # replicate this column just after the other messages for easy comparison

duplication_disagreement.to_csv('duplication_examination.csv')

## Agreement on dismissal

In [ ]:
# pre-process to remove None
dism_agree_check = dupclean_praisecheck.copy()

for k in range(1,NUMBER_OF_QUANTIFIERS_PER_PRAISE+1):
    col = 'DISMISSED '+str(k)
    dism_agree_check.loc[dism_agree_check[col].isnull(),col]=0


dism_agree_check['DISMISSAL AGREED'] = [len(set(kk))==1 for kk in dism_agree_check[col_dismissed].values]

dismiss_disagreement = dism_agree_check.loc[dism_agree_check['DISMISSAL AGREED']==False,:]
dismiss_disagreement= dismiss_disagreement.drop(col_scores+col_dupids+['SPREAD'],axis=1)

In [ ]:

dismiss_disagreement['ORIGINAL MSG']=dismiss_disagreement['REASON'] # replicate this column just after the other messages for easy comparison

dismiss_disagreement.to_csv('dismissal_disagreed.csv')

print(f'Among {len(dism_agree_check)} praises, {len(dismiss_disagreement)} ({len(dismiss_disagreement)/len(praisecheck_df)*100:.2f}%) do not agree on dismissal')



Praise instances with disagreements in dismissal are collected in'results/dismissal_disaggreed.csv'. You can further look into who dismissed and who did not. 

